In [14]:
from openai import OpenAI
from tqdm import tqdm
import json
import numpy as np
import random
from collections import Counter
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_chinese import Rouge
import jieba
import json
from bert_score import score
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from GeneralRequestProcessor import GeneralRequestProcessor

In [15]:
# 读取和保存数据的函数
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"错误：找不到文件 {file_path}。请检查文件路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"错误：无法解码文件 {file_path} 中的JSON数据。请确保这是一个有效的JSON文件。")
        return None
    
def save_to_json_file(data, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        print(f"数据成功保存到 {file_path}")
    except Exception as e:
        print(f"保存文件时发生错误: {e}")   
## 获取8-shot
def make_shot(origin_data=None):
    shot_8 = ""
    for index, single_data in enumerate(origin_data[4492:4500]):
        shot_ = f"【**案例{index}**：\n{single_data['conversations'][1]['value']}\n】"
        shot_8 += shot_
    return shot_8

# 读取原始数据并处理成需要的格式
def make_messages(origin_data=None):
    """
    将原始数据集转换为大模型微调所需数据格式的新数据集
    """
    messages = []
    Real_outputs = []
    shot_8 = make_shot(origin_data)
    for index, single_data in enumerate(origin_data):
        random_number = random.randint(0, 9)
        message = [
            {"role": "system", "content": ""},
#             {"role": "user", "content": single_data['conversations'][0]['value']}
            {"role": "user", "content": "{}\n-{},以下是供你参考的人文社科领域学术文本摘要案例\n{}".format(head_template_COT3, single_data['conversations'][0]['value'], shot_8)}
        ]
        messages.append([message,index])
        Real_outputs.append(single_data['conversations'][1]['value'])
            
    return messages,Real_outputs

def ChatCompletions(message):
    response = client.chat.completions.create(
        model="MyModel",
        messages=message,
        stream=False,
        temperature=0.01,
        max_tokens=5000,
    )
    return response.choices[0].message.content        

def process_cot(abstract, model):
    abstract_COT = []
    if model == 'qwen':
        count = []
        for idx, value in enumerate(abstract):
            try:
                Model_output_COT = value.split('<最终摘要>')[1].strip('\n')
            except:
                count.append(idx)
                Model_output_COT = value
            abstract_COT.append(Model_output_COT)
        print(f"本次存在问题的id为{count}", '\n', '------------------------------')
    elif model == 'llama':
        count = []
        for idx, value in enumerate(abstract):
            try:
                Model_output_COT = value.split('最终摘要')[1].strip('\n')
            except:
                count.append(idx)
                Model_output_COT = value.split('\n')[-1]
            abstract_COT.append(Model_output_COT)
        print(f"本次存在问题的id为{count}", '\n', '------------------------------')
    else:
        print("输入有误")
    return abstract_COT

def process_deep(abstract):
    abstract_new = []
    idx_list = []
    count = []
    for idx, value in enumerate(abstract):
        try:
            Model_output_deepseek = value.split('</think>')[1].strip('\n')
        except:
            Model_output_deepseek = "文本生成失败"
        # 测试长度
        encoded_text = tokenizer.encode(value)
        if len(encoded_text) > 3000:
            Model_output_deepseek = "文本生成失败"
            count.append(idx)
        else:
            idx_list.append(idx)
        abstract_new.append(Model_output_deepseek)
    print(f"本次存在问题的id为{count}", '\n', '------------------------------')
    return abstract_new, idx_list

head_template = [
    "请你根据给出的学术全文本内容，生成关于该文本的摘要。",
    "请根据提供的学术文本内容，生成一个该文本的摘要。",
    "请根据给定的学术全文，撰写一个简要总结。",
    "请根据所提供的学术文章内容，生成一个概述。",
    "你需要阅读以下学术文本，并根据其内容编写摘要",
    "根据给出的学术材料，请你整理出一份摘要。",
    "请将下面的学术文本内容总结成一个简短的摘要。",
    "请从提供的学术文本中提炼出摘要内容。",
    "请根据以下学术文本写一段简短的总结。",
    "请基于下列学术全文，提炼并生成相关摘要。"
]
head_template_COT = """
下面，我将会给你一份学术文本内容，你需要分析给出的内容，编写关于该学术文本的摘要。除去学术文本，我还会给你其他学术文本的摘要作为参考案例
## 编写摘要要求
- 首先你需要分析我给你的学术文本摘要案例，案例总共8个，你需要分析每一份摘要案例的行文特点和语言特征，作为你生成摘要的支撑。
- 然后你需要分析我所给你的学术文本主要讲述的核心内容，涉及的实验对象等学术信息，并分析你认为的每一个段落的关键语句。
- 最后，你需要根据你的分析，结合你从我所提供的摘要的行文特点和语言特征，编写一份我所给你的学术文本内容的摘要。
## 输出格式
- 你输出数据的格式应严格为：<所给摘要特征总结>xxxxx\n<学术文本内容分析>xxxxx\n<最终摘要>xxxxx
"""

head_template_COT2 = """
下面，我将会给你一份学术文本内容，你需要分析给出的内容，编写关于该学术文本的摘要。除去学术文本，我还会给你其他学术文本的摘要作为参考案例
## 编写摘要要求
- 首先你需要分析我给你的学术文本摘要案例，案例总共8个，你需要分析每一份摘要案例的行文特点和语言特征，作为你生成摘要的支撑。
- 然后你需要分析我所给你的学术文本主要讲述的核心内容，涉及的实验对象等学术信息，并分析你认为的每一个段落的关键语句。
- 最后，你需要根据你的分析，结合你从我所提供的摘要的行文特点和语言特征，编写一份我所给你的学术文本内容的摘要。
## 输出格式
- 你的最终输出只需要给出你的摘要即可，不用给出其他内容！
"""

head_template_COT3 = """
下面，我会先给你一份人文社科学术文本，你需要总结该文本摘要。此外，我还会给你8个摘要案例作为编写学术文本的参考，编写中需要注意的点如下：
## 所给摘要特征总结要求
- 你需要分析我给你的学术文本摘要案例，案例总共8个，你需要分析每一份摘要案例的行文特点和语言特征，作为你生成摘要的支撑。
- 你要根据你对案例摘要分析的内容，在<所给摘要特征总结>模块给出人文社科领域学术文本摘要的行文特点和语言特征，为之后修改摘要提供依据。
## 学术文本内容分析要求
- 人文社科领域摘要的重点在于背景描述和本文论点的结合。
- 首先你需要识别出本文的研究对象，并在<学术文本内容分析>模块中以【研究对象：xxxx】的格式给出。
- 其次你需要识别出本文的研究背景，并在<学术文本内容分析>模块中以【研究背景：xxxx】的格式给出。
- 然后你需要以文章段落为单位进行分析，找出文中关于研究对象的论述，并在<学术文本内容分析>模块中以【论述x：xxxx】的格式给出。
- 你需要分析不同论述之间是并列、递进亦或对比等关系，并在<学术文本内容分析>模块中按照论述关系三元组【论述m，关系，论述n】的格式给出。
- 最后你要根据总结的论述关系三元组，分析出本文论述的核心论点，并在<学术文本内容分析>模块中以【核心论点：xxxx】的格式给出
## 初步摘要的编写
- 你要围绕<学术文本内容分析>模块中给出的“研究对象”、“研究背景”以及“核心论点”部分内容编写初步摘要，并在<初步摘要>模块给出。
## 摘要审改的编写
- 根据<初步摘要>模块的初步摘要，结合<所给摘要特征总结>模块中人文社科领域学术文本的行文特点和语言特征，以及你得到的8个案例，对初步摘要进行修改。
- 你需要比对初步摘要同案例摘要在语言特征方面的区别，并在<摘要审改>模块中以【区别x:xxxxx】形式给出。
- 你需要根据你所罗列的区别信息，在<摘要审改>模块中给出摘要修改的思路。
## 最终摘要的编写
- 根据在<摘要审改>模块中得到的思路，对<初步摘要>模块中的摘要内容进行修改，力求修改后的语言和结构特征同真实摘要一致。
- 真实摘要中较少使用（首先、其次、再次）这种前后顺序的连接词，因此在最终摘要编写过程中要注意该语言表达特点。
- 注意最终摘要中的组织语句主语、谓语宾语等结构的语言表达习惯要同真实摘要相一致。在<最终摘要>模块中给出你所转写的最终摘要。
## 输出格式
- 你输出数据的格式应严格为：<所给摘要特征总结>xxxxx\n<学术文本内容分析>xxxxx\n<初步摘要>xxxxx\n<摘要审改>xxxxx\n<最终摘要>xxxxx
- <最终摘要>模块仅需要给出你修改好的最终摘要，不要再输出其他的内容！
"""

In [16]:
tokenizer = AutoTokenizer.from_pretrained('/gemini/data-2/DeepSeek-R1-Distill-llama-8B/')
# 使用函数读取JSON文件
file_path = '/gemini/code/Abstract_extract/ProcessedData/rdfybk/train_data_abstract_mulIns_small.json'  # 将这里的文件路径替换为你自己的JSON文件路径
train_test_data = read_json_file(file_path)
# 获取输入数据
messages,Real_outputs = make_messages(train_test_data)

# 读取所有的摘要
abstract1 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B.json")
abstract2 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_cot.json")
abstract3 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_1.2.json")
abstract4 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B.json")
abstract5 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B_cot2.json")
abstract6 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B_1.4.json")
abstract7 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_8B.json")
abstract8 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_8B_cot2.json")
abstract9 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_7B.json")
abstract10 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_7B_cot.json")
# 新补充的实验
abstract11 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_cot_plus2.json")
abstract12 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_2.1.json")
abstract13 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B_cot_plus2.json")

In [17]:
abstract2_COT = process_cot(abstract2, 'qwen')
abstract5_COT = process_cot(abstract5, 'llama')
abstract7_new, idx_list = process_deep(abstract7)
abstract8_new, idx_list = process_deep(abstract8)
abstract9_new, idx_list = process_deep(abstract9)
abstract10_new, idx_list = process_deep(abstract10)
abstract11_COT = process_cot(abstract11, 'qwen')
abstract12_COT = process_cot(abstract12, 'qwen')
abstract13_COT = process_cot(abstract13, 'llama')

本次存在问题的id为[21, 26, 141, 229, 232, 239, 270, 335, 357, 497] 
 ------------------------------
本次存在问题的id为[0, 40, 62, 81, 83, 103, 178, 191, 195, 219, 223, 225, 361, 371, 387, 395, 412, 436, 452, 469, 477, 487, 489, 497, 499] 
 ------------------------------
本次存在问题的id为[21, 25, 26, 33, 36, 42, 106, 126, 128, 135, 141, 145, 155, 169, 198, 205, 215, 224, 229, 232, 239, 266, 270, 271, 295, 298, 320, 333, 335, 367, 385, 420, 443, 469, 477, 489, 497] 
 ------------------------------
本次存在问题的id为[2, 35, 83, 169, 229, 464, 469, 476, 482, 488, 496, 497, 499] 
 ------------------------------
本次存在问题的id为[357] 
 ------------------------------
本次存在问题的id为[36, 139, 486, 497] 
 ------------------------------
本次存在问题的id为[] 
 ------------------------------
本次存在问题的id为[17, 65, 120, 136, 211, 213, 273, 277, 328, 331, 369, 378, 390, 407, 409, 466, 497] 
 ------------------------------
本次存在问题的id为[70, 79, 84, 92, 187, 230, 250, 336, 374, 380, 465, 470, 485] 
 ------------------------------


In [20]:
hyps = []
refs = []
for idx in range(len(abstract2_COT)):
    hyps.append(list(jieba.cut(abstract11_COT[idx])))
    refs.append(list(jieba.cut(Real_outputs[4500+idx])))

In [21]:
# 计算多个候选文本的 BLEU 分数
bleu_scores = []
for ref, hyp in zip(refs, hyps):
    score = sentence_bleu([ref], hyp)
    bleu_scores.append(score)
# for idx, score in enumerate(bleu_scores):
#     print(f"候选文本 {idx + 1} 的 BLEU 分数: {score}")
np.mean(bleu_scores)

0.09734364841609582